In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import glob

# Classify using reservoir causality data

In [2]:
# load computed causality matrices
con_files = [f for f in glob.glob("data/sub-CON*.npy")]
pat_files = [f for f in glob.glob("data/sub-PAT*.npy")]

# filter out excluded subjects
excluded = ["PAT08", "CON03", "PAT03", "PAT11"]
con_files = [f for f in con_files if f[9:14] not in excluded]
pat_files = [f for f in pat_files if f[9:14] not in excluded]

# transform for LDA
con_causalities = np.array([np.load(f) for f in con_files])
pat_causalities = np.array([np.load(f) for f in pat_files])
con_flattened = con_causalities.reshape((con_causalities.shape[0], -1))
pat_flattened = pat_causalities.reshape((pat_causalities.shape[0], -1))

In [3]:
from scipy.stats import entropy

In [5]:
entropy_control = []
for elem in con_flattened:
    pd_series = pd.Series(elem)
    counts = pd_series.value_counts()
    entr = entropy(counts)
    entropy_control.append(entr)

In [6]:
entropy_patient = []
for elem in pat_flattened:
	pd_series = pd.Series(elem)
	counts = pd_series.value_counts()
	entr = entropy(counts)
	entropy_patient.append(entr)

In [7]:
np.var(entropy_control), np.var(entropy_patient)

(0.0001730633072656175, 9.06879846894294e-05)

In [8]:
import pingouin as pg
pg.ttest(entropy_patient, entropy_control)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.039417,18,two-sided,0.312372,"[-0.01, 0.02]",0.464841,0.579,0.166327


In [9]:


con_files = [f for f in glob.glob("data/sub-CON*.npy")]
pat_files = [f for f in glob.glob("data/sub-PAT*.npy")]

con_fmri_files = [f[:-21] + ".tsv" for f in con_files]
pat_fmri_files = [f[:-21] + ".tsv" for f in pat_files]

con_corr_matrices = []
for filename in con_fmri_files:
	df = pd.read_csv(filename, header=None)
	data = np.array(df)
	correlation_matrix = np.corrcoef(data.T)
	con_corr_matrices.append(correlation_matrix)
pat_corr_matrices = []
for filename in pat_fmri_files:
	df = pd.read_csv(filename, header=None)
	data = np.array(df)
	correlation_matrix = np.corrcoef(data.T)
	pat_corr_matrices.append(correlation_matrix)

# flatten for LDA
con_correlations = np.array(con_corr_matrices)
pat_correlations = np.array(pat_corr_matrices)
con_flattened_corr = con_correlations.reshape((con_correlations.shape[0], -1))
pat_flattened_corr = pat_correlations.reshape((pat_correlations.shape[0], -1))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [10]:
entropy_control_corr = []
for elem in con_flattened_corr:
	pd_series = pd.Series(elem)
	counts = pd_series.value_counts()
	entr = entropy(counts)
	entropy_control_corr.append(entr)

In [11]:
entropy_patient_corr = []
for elem in pat_flattened_corr:
	pd_series = pd.Series(elem)
	counts = pd_series.value_counts()
	entr = entropy(counts)
	entropy_patient_corr.append(entr)

In [12]:
pg.ttest(entropy_patient_corr, entropy_control_corr)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.080397,21.810364,two-sided,0.291768,"[-0.01, 0.03]",0.439483,0.57,0.176677
